### Workbook for extracting WorldPop population estimates for varying bridge catchment distances, reading in WorldPop rasters for Rwanda and surrounding countries
Week of May 12, 2025
Author: Adele Birkenes

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import GeoJson

Task 1: Read in data on bridge sites and village boundaries

Note: The village boundaries are not used in this analysis, but they contain the custom Rwanda Transverse Mercator projection that will be copied over to the bridge sites and WorldPop data

In [2]:
synced_path = "../../synced-data/population-exploration/"
unsynced_path = "../../unsynced-data"

# Village boundaries (geodataframe)
village_boundaries_fp = os.path.join(synced_path,"Rwanda Village Boundaries/Village.shp")
Rwanda_village_boundaries = gpd.read_file(village_boundaries_fp)

# Bridge sites (dataframe)
bridge_sites_fp = os.path.join(synced_path, "Rwanda Sites with All Population Fields_Exported 2025.04.11.csv")
bridge_sites = pd.read_csv(bridge_sites_fp, encoding='ISO-8859-1') # Note: This encoding accommodates special characters

In [3]:
# Convert bridge sites dataframe to geodataframe that has custom Rwanda TM CRS copied from village boundaries geodataframe
def map_bridges(bridges, bridges_lat, bridges_lon, village_boundaries):

    # Check CRS of village boundaries gdf
    print(f'The CRS of the village boundaries gdf is: {village_boundaries.crs}')

    # Create lat/lon variables
    lon = bridges[bridges_lon]
    lat = bridges[bridges_lat]

    # Create gdf of bridges data by converting lat/lon values to list of Shapely Point objects
    bridge_points = gpd.GeoDataFrame(bridges, geometry=gpd.points_from_xy(x=lon, y=lat), crs='EPSG:4326')

    # Set CRS of bridges gdf to CRS of village boundaries gdf
    bridge_points.to_crs(village_boundaries.crs, inplace=True)

    # Check that reprojection was successful
    print(f'The CRS of the bridges gdf is: {bridge_points.crs}')
    
    return bridge_points

bridge_points = map_bridges(bridges = bridge_sites,
                            bridges_lat = "Bridge Opportunity: GPS (Latitude)",
                            bridges_lon = "Bridge Opportunity: GPS (Longitude)",
                            village_boundaries = Rwanda_village_boundaries)

The CRS of the village boundaries gdf is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
The CRS of the bridges gdf is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_

Task 2: Read in WorldPop rasters for Rwanda and surrounding countries. Mosaic rasters and reproject to custom Rwanda Transverse Mercator projection.

In [20]:
# Create list of WorldPop raster files for Rwanda and surrounding countries: DRC, Uganda, Tanzania, and Burundi
countries = ["bdi", "cod", "rwa", "uga", "tza"]
WorldPop_2020 = [os.path.join(unsynced_path, f"WorldPop_Rasters_2020.nosync/{country}_ppp_2020_UNadj_constrained.tif") for country in countries]

# Mosaic WorldPop rasters in list
WorldPop_2020_mosaic, mosaic_transform = merge([rasterio.open(raster) for raster in WorldPop_2020])

# Specify output file path for the mosaic
mosaic_fp = os.path.join(unsynced_path, "WorldPop_Rasters_2020.nosync/worldpop_2020_mosaic.tif")

# Calculate width and height from the shape of the mosaic array
height, width = WorldPop_2020_mosaic.shape[1], WorldPop_2020_mosaic.shape[2]

# Save the mosaic to a GeoTIFF file
with rasterio.open(
    mosaic_fp, 
    'w', 
    driver='GTiff', 
    height=height, 
    width=width, 
    count=1, 
    dtype=WorldPop_2020_mosaic.dtype, 
    crs="EPSG:4326",
    transform=mosaic_transform
) as dst:
    dst.write(WorldPop_2020_mosaic[0], 1)

In [ ]:
# Reproject raster mosaic to custom Rwanda TM CRS
def reproject_raster(population_raster_file, villages_boundaries):
    
    # Open the raster file
    with rasterio.open(population_raster_file) as src:
        src_crs = src.crs
        dst_crs = villages_boundaries.crs
        print(f'The starting CRS of the raster is: {src_crs}')
        print(f'The target CRS from villages is: {dst_crs}')

        # Calculate the transform, width, height for the destination raster
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )

        # Update metadata
        meta = src.meta.copy()
        meta.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Prepare an empty destination array
        dest = np.empty((src.count, height, width), dtype=src.dtypes[0])

        # Reproject
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=dest[i - 1],
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )

    # Save to file
    reprojected_raster_file = os.path.join(unsynced_path, "WorldPop_Rasters_2020.nosync/reprojected_worldpop_2020_mosaic.tif")
    with rasterio.open(reprojected_raster_file, 'w', **meta) as dst:
        dst.write(dest)

    print(f'Reprojected raster saved to: {reprojected_raster_file}')
    
    return dest, transform, meta

reprojected_worldpop = reproject_raster(population_raster_file=mosaic_fp,
                                        villages_boundaries=Rwanda_village_boundaries)

The starting CRS of the raster is: EPSG:4326
The target CRS from villages is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Reprojected raster saved to: ../../unsynced-data/WorldPop_Rasters_2020.nosync/reprojected_worldpop_2020_mosaic.tif


Task 3: Create 1, 1.5, 2, and 3 km buffers around each bridge site

In [4]:
def create_bridge_buffers(bridges, buffer_distance):

    # Create buffers around bridges according to user's distance input
    buf = bridges.geometry.buffer(distance=buffer_distance)

    # Add bridges point attribute information to buffers
    bridge_buffers = bridge_points.copy()
    bridge_buffers['geometry'] = buf

    return bridge_buffers

bridge_buffers_1km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 1000)
bridge_buffers_1_5km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 1500)
bridge_buffers_2km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 2000)
bridge_buffers_3km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 3000)

In [17]:
# Create a base map
buffer_map = folium.Map(location=[-2.0, 30.0], zoom_start=8)

# Map bridge points and buffers
GeoJson(bridge_buffers_1km, name="1 km Buffers").add_to(buffer_map)
GeoJson(bridge_buffers_1_5km, name="1.5 km Buffers").add_to(buffer_map)
GeoJson(bridge_buffers_2km, name="2 km Buffers").add_to(buffer_map)
GeoJson(bridge_buffers_3km, name="3 km Buffers").add_to(buffer_map)

# Add layer control to toggle between layers
folium.LayerControl().add_to(buffer_map)

# Display the map
#buffer_map

Task 4: Extract population estimates from WorldPop raster for each buffer

In [34]:
def extract_pop(reprojected_pop_raster, population_raster_name, villages_near_bridges):
    
    # Copy input geodataframe to avoid modifying the original
    villages_copy = villages_near_bridges.copy()
    
    with rasterio.open(reprojected_pop_raster) as src:
        # Print information from raster profile
        print("CRS:", src.crs)
        print("Raster shape:", src.shape)
        print("Number of Bands:", src.count)

        # Iterate over each village and extract population data
        results = []
        for idx, row in villages_copy.iterrows():
            try:
                out_image = mask(src, [row.geometry], crop=True)
                out_image = out_image[0]

                # Handle masked arrays and replace with 0
                if np.ma.is_masked(out_image):
                    out_image = out_image.filled(0)

                # Remove negative values (e.g., nodata placeholders)
                out_image = np.where(out_image < 0, 0, out_image)

                total_population = np.sum(out_image)

            except Exception as e:
                print(f"Error at village {idx}: {e}")
                total_population = 0  # or np.nan

            results.append(total_population)

        # Add population data to the gdf
        villages_copy[population_raster_name] = results

    return villages_copy

# Get file path for reprojected raster
reprojected_raster_file = os.path.join(unsynced_path, "WorldPop_Rasters_2020.nosync/reprojected_worldpop_2020_mosaic.tif")

# Extract population estimates for each buffer distance
pop_estimates_1km = extract_pop(reprojected_pop_raster = reprojected_raster_file,
                                population_raster_name = "WorldPop_1km",
                                villages_near_bridges = bridge_buffers_1km)

pop_estimates_1_5km = extract_pop(reprojected_pop_raster = reprojected_raster_file,
                                  population_raster_name = "WorldPop_1_5km",
                                  villages_near_bridges = bridge_buffers_1_5km)

pop_estimates_2km = extract_pop(reprojected_pop_raster = reprojected_raster_file,
                                population_raster_name = "WorldPop_2km",
                                villages_near_bridges = bridge_buffers_2km)

pop_estimates_3km = extract_pop(reprojected_pop_raster = reprojected_raster_file,
                                population_raster_name = "WorldPop_3km",
                                villages_near_bridges = bridge_buffers_3km)

CRS: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Raster shape: (23438, 34126)
Number of Bands: 1
Error at village 197: Input shapes do not overlap raster.
Error at village 198: Input shapes do not overlap raster.
Error at village 459: tuple index out of range
Error at village 542: tuple index out of range
Error at village 548: Input shapes do not overlap raster.
Error at village 733: tuple index out of range
Error at village 734: tuple index out of range
Error at village 735: tuple

In [35]:
# Combine population estimates into a single geodataframe
pop_estimates_combined = bridge_points.copy()

# Add population estimates from each buffer to the combined geodataframe
# Note: The merge is done on the index, which is the same for all geodataframes
pop_estimates_combined = pop_estimates_combined.merge(
    pop_estimates_1km[['WorldPop_1km']], left_index=True, right_index=True
).merge(
    pop_estimates_1_5km[['WorldPop_1_5km']], left_index=True, right_index=True
).merge(
    pop_estimates_2km[['WorldPop_2km']], left_index=True, right_index=True
).merge(
    pop_estimates_3km[['WorldPop_3km']], left_index=True, right_index=True
)

In [36]:
# Update the population estimates to replace 0 with NaN
# Note: The zeroes are because a number of villages are outside of Rwanda and its neighboring countries (the raster extent)
pop_estimates_combined['WorldPop_1km'] = pop_estimates_combined['WorldPop_1km'].replace(0, np.nan)
pop_estimates_combined['WorldPop_1_5km'] = pop_estimates_combined['WorldPop_1_5km'].replace(0, np.nan)
pop_estimates_combined['WorldPop_2km'] = pop_estimates_combined['WorldPop_2km'].replace(0, np.nan)
pop_estimates_combined['WorldPop_3km'] = pop_estimates_combined['WorldPop_3km'].replace(0, np.nan)

In [37]:
# Export the combined geodataframe as a CSV file
pop_estimates_combined_fp = os.path.join(synced_path, "Rwanda_Bridge_Sites_WorldPop_2020_Estimates_Projected.csv")
#pop_estimates_combined.to_csv(pop_estimates_combined_fp, index=False)

Task 5: Calculate summary statistics

In [38]:
# Calculate mean population for each distance radius
def mean_pop(pop_estimates_combined, pop_column):
    mean_value = pop_estimates_combined[pop_column].mean()
    print(f"The mean of {pop_column} is: {mean_value:.2f}")
    return mean_value

mean_pop(pop_estimates_combined, 'WorldPop_1km')
mean_pop(pop_estimates_combined, 'WorldPop_1_5km')
mean_pop(pop_estimates_combined, 'WorldPop_2km')
mean_pop(pop_estimates_combined, 'WorldPop_3km')
mean_pop(pop_estimates_combined, "Bridge Opportunity: Population Estimate 2000m")

The mean of WorldPop_1km is: 2838.88
The mean of WorldPop_1_5km is: 6222.52
The mean of WorldPop_2km is: 10766.41
The mean of WorldPop_3km is: 23241.60
The mean of Bridge Opportunity: Population Estimate 2000m is: 9769.34


np.float64(9769.340919269298)

In [39]:
# Calculate mean population for sites with value "Rwanda Full Scale Research - Randomized but Not Baselined"
# OR "Rwanda Full Scale Research - Randomized and Baselined - Long Term Control" in the Research Initiative field
print("Mean population for sites with research initiatives:")

# Define research values of interest
research_values = ["Rwanda Full Scale Research - Randomized but Not Baselined",
                   "Rwanda Full Scale Research - Randomized and Baselined - Long Term Control"]

# Calculate mean population for the defined research values
mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_1km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_1_5km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_2km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_3km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "Bridge Opportunity: Population Estimate 2000m"
)

Mean population for sites with research initiatives:
The mean of WorldPop_1km is: 2236.09
The mean of WorldPop_1_5km is: 5273.33
The mean of WorldPop_2km is: 9315.01
The mean of WorldPop_3km is: 19277.88
The mean of Bridge Opportunity: Population Estimate 2000m is: 7044.64


np.float64(7044.636339285715)

Task 6: Rerun calculations in unprojected CRS (WGS84)

In [40]:
# Update extract pop function to work with unprojected raster (converts buffer gdf to match raster CRS)
def extract_pop_unproj(pop_raster, population_raster_name, villages_near_bridges):
    
    # Copy input geodataframe to avoid modifying the original
    villages_copy = villages_near_bridges.copy()
    
    with rasterio.open(pop_raster) as src:
        # Print information from raster profile
        print("Raster CRS:", src.crs)
        print("Raster shape:", src.shape)
        print("Number of Bands:", src.count)

        # Reproject buffer gdf to match raster CRS and print new CRS
        villages_copy = villages_copy.to_crs(src.crs)
        print("Buffer CRS:", villages_copy.crs)

        # Iterate over each village and extract population data
        results = []
        for idx, row in villages_copy.iterrows():
            try:
                out_image = mask(src, [row.geometry], crop=True)
                out_image = out_image[0]

                # Handle masked arrays and replace with 0
                if np.ma.is_masked(out_image):
                    out_image = out_image.filled(0)

                # Remove negative values (e.g., nodata placeholders)
                out_image = np.where(out_image < 0, 0, out_image)

                total_population = np.sum(out_image)

            except Exception as e:
                print(f"Error at village {idx}: {e}")
                total_population = 0  # or np.nan

            results.append(total_population)

        # Add population data to the gdf
        villages_copy[population_raster_name] = results

    return villages_copy

# Get file path for unprojected raster
unprojected_raster_file = os.path.join(unsynced_path, "WorldPop_Rasters_2020.nosync/worldpop_2020_mosaic.tif")

# Extract population estimates for each buffer distance
pop_estimates_1km_unproj = extract_pop_unproj(pop_raster = unprojected_raster_file,
                                population_raster_name = "WorldPop_1km",
                                villages_near_bridges = bridge_buffers_1km)

pop_estimates_1_5km_unproj = extract_pop_unproj(pop_raster = unprojected_raster_file,
                                  population_raster_name = "WorldPop_1_5km",
                                  villages_near_bridges = bridge_buffers_1_5km)

pop_estimates_2km_unproj = extract_pop_unproj(pop_raster = unprojected_raster_file,
                                population_raster_name = "WorldPop_2km",
                                villages_near_bridges = bridge_buffers_2km)

pop_estimates_3km_unproj = extract_pop_unproj(pop_raster = unprojected_raster_file,
                                population_raster_name = "WorldPop_3km",
                                villages_near_bridges = bridge_buffers_3km)

Raster CRS: EPSG:4326
Raster shape: (22610, 33886)
Number of Bands: 1
Buffer CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Error at village 197: Input shapes do not overlap raster.
Error at village 198: Input shapes do not overlap raster.
Error at village 459: tuple index out of range
Error at village 542: tuple index out of range
Error at village 548: Input shapes do not overlap raster.
Error at village 733: tuple index out of range
Error at village 734: tuple index out of range
Error at village 735: tuple index out of range
Error at village 736: tuple index out of range
Error at village 923: tuple index out of range
Error at village 1077: tuple index out of range
Error at village 1274: tuple index out of range
Error at village

In [41]:
# Combine population estimates into a single geodataframe
pop_estimates_combined_unproj = bridge_points.copy()

# Add population estimates from each buffer to the combined geodataframe
# Note: The merge is done on the index, which is the same for all geodataframes
pop_estimates_combined_unproj = pop_estimates_combined_unproj.merge(
    pop_estimates_1km_unproj[['WorldPop_1km']], left_index=True, right_index=True
).merge(
    pop_estimates_1_5km_unproj[['WorldPop_1_5km']], left_index=True, right_index=True
).merge(
    pop_estimates_2km_unproj[['WorldPop_2km']], left_index=True, right_index=True
).merge(
    pop_estimates_3km_unproj[['WorldPop_3km']], left_index=True, right_index=True)

# Update the population estimates to replace 0 with NaN
pop_estimates_combined_unproj['WorldPop_1km'] = pop_estimates_combined_unproj['WorldPop_1km'].replace(0, np.nan)
pop_estimates_combined_unproj['WorldPop_1_5km'] = pop_estimates_combined_unproj['WorldPop_1_5km'].replace(0, np.nan)
pop_estimates_combined_unproj['WorldPop_2km'] = pop_estimates_combined_unproj['WorldPop_2km'].replace(0, np.nan)
pop_estimates_combined_unproj['WorldPop_3km'] = pop_estimates_combined_unproj['WorldPop_3km'].replace(0, np.nan)

In [42]:
# Calculate mean population for each distance radius
def mean_pop(pop_estimates_combined_unproj, pop_column):
    mean_value = pop_estimates_combined_unproj[pop_column].mean()
    print(f"The mean of {pop_column} is: {mean_value:.2f}")
    return mean_value

mean_pop(pop_estimates_combined_unproj, 'WorldPop_1km')
mean_pop(pop_estimates_combined_unproj, 'WorldPop_1_5km')
mean_pop(pop_estimates_combined_unproj, 'WorldPop_2km')
mean_pop(pop_estimates_combined_unproj, 'WorldPop_3km')
mean_pop(pop_estimates_combined_unproj, "Bridge Opportunity: Population Estimate 2000m")

The mean of WorldPop_1km is: 2887.39
The mean of WorldPop_1_5km is: 6322.43
The mean of WorldPop_2km is: 10949.84
The mean of WorldPop_3km is: 23639.14
The mean of Bridge Opportunity: Population Estimate 2000m is: 9769.34


np.float64(9769.340919269298)

In [43]:
# Calculate mean population for sites with value "Rwanda Full Scale Research - Randomized but Not Baselined"
# OR "Rwanda Full Scale Research - Randomized and Baselined - Long Term Control" in the Research Initiative field
print("\nMean population for sites with research initiatives:")

# Define research values of interest
research_values = ["Rwanda Full Scale Research - Randomized but Not Baselined",
                   "Rwanda Full Scale Research - Randomized and Baselined - Long Term Control"]

# Calculate mean population for the defined research values
mean_pop(
    pop_estimates_combined_unproj[
        pop_estimates_combined_unproj['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_1km"
)

mean_pop(
    pop_estimates_combined_unproj[
        pop_estimates_combined_unproj['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_1_5km"
)

mean_pop(
    pop_estimates_combined_unproj[
        pop_estimates_combined_unproj['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_2km"
)

mean_pop(
    pop_estimates_combined_unproj[
        pop_estimates_combined_unproj['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_3km"
)

mean_pop(
    pop_estimates_combined_unproj[
        pop_estimates_combined_unproj['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "Bridge Opportunity: Population Estimate 2000m"
)


Mean population for sites with research initiatives:
The mean of WorldPop_1km is: 2294.09
The mean of WorldPop_1_5km is: 5377.69
The mean of WorldPop_2km is: 9518.26
The mean of WorldPop_3km is: 19632.49
The mean of Bridge Opportunity: Population Estimate 2000m is: 7044.64


np.float64(7044.636339285715)

In [44]:
# Combine the projected and unprojected population estimates into a single geodataframe
pop_estimates_combined = pop_estimates_combined.merge(
    pop_estimates_combined_unproj[['WorldPop_1km', 'WorldPop_1_5km', 'WorldPop_2km', 'WorldPop_3km']],
    left_index=True, right_index=True, suffixes=('', '_unproj')
)

In [45]:
# Export the combined geodataframe as a CSV file
pop_estimates_proj_unproj_fp = os.path.join(synced_path, "Rwanda_Bridge_Sites_WorldPop_2020_with3km.csv")
pop_estimates_combined.to_csv(pop_estimates_proj_unproj_fp, index=False)


### Projected Summary Statistics

**Mean population for all sites:**
- The mean of WorldPop_1km is: 2838.88
- The mean of WorldPop_1_5km is: 6222.52
- The mean of WorldPop_2km is: 10766.41
- The mean of WorldPop_3km is: 23241.60
- The mean of Salesforce WorldPop estimate at 2km is: 9769.34

**Mean population for sites with research initiatives:**
- The mean of WorldPop_1km is: 2236.09
- The mean of WorldPop_1_5km is: 5273.33
- The mean of WorldPop_2km is: 9315.01
- The mean of WorldPop_3km is: 19277.88
- The mean of Salesforce WorldPop estimate at 2km is: 7044.64

### Unprojected Summary Statistics

**Mean population for all sites:**
- The mean of WorldPop_1km is: 2887.39
- The mean of WorldPop_1_5km is: 6322.43
- The mean of WorldPop_2km is: 10949.84
- The mean of WorldPop_3km is: 23639.14

**Mean population for sites with research initiatives:**
- The mean of WorldPop_1km is: 2294.09
- The mean of WorldPop_1_5km is: 5377.69
- The mean of WorldPop_2km is: 9518.26
- The mean of WorldPop_3km is: 19632.49
